# Limpando os dados do TSE

[Voltar ao Índice](00_indice.ipynb)

Esse notebook pega como input os dados brutos do TSE e os limpa e padroniza (coloca os dados de todos os anos, de 2004 a 2020, no mesmo formato e estrutura).

Note que, nos casos de anos passados nos quais os dados eram disponibilizados sem cabeçalho, esse processo depende da existência de arquivos `header.txt` nos diretórios dos arquivos brutos. Esse arquivo não vem com os dados brutos, e foi criado com o script `../scripts/criaHeader.py` e os arquivos com a lista de colunas no diretório `../dados/aux/`.

In [1]:
import os
import pandas as pd
import numpy as np
import re
import csv
from glob import glob

import utils as xu

## Funções de plotagem

Para testar e comparar a composição das colunas. Foram movidas para o arquivo `plot_comparison.py`. Isso pode causar problemas abaixo, mas basta carregar o módulo e usá-lo para resolver o problema.

## Funções de carregar e preparar dados

In [2]:
def load_tse_file(filename, header=None, conv=None, verbose=False):
    """
    Given a data file from TSE and a header (required if file does not 
    have a header, and this happens when the file has a .txt extension),
    load it.
    """
    
    if filename[-4:] == '.txt' and type(header) == type(None):
        if verbose:
            print('Will load header.')
        data_dir = '/'.join(filename.split('/')[:-1])
        header   = pd.read_csv(data_dir + '/header.txt', sep=';').columns
    
    # Load TSE file:
    df = pd.read_csv(filename, sep=';', encoding='latin1', names=header, low_memory=False, converters=conv)
    
    return df


def substr_in_list(str_list, substr, not_in=False):
    """
    Return the items in list `str_list` that contains `substr` (str).
    """
    if not_in:
        sel_str = list(filter(lambda s: s.find(substr) == -1, str_list))
    else:
        sel_str = list(filter(lambda s: s.find(substr) != -1, str_list))
    return sel_str


def n_substr_in_list(str_list, substr):
    """
    Return the number of strings in list `str_list` that contains `substr` (str).
    """
    sel_str = substr_in_list(str_list, substr)
    return len(sel_str)


def substr_in_list_Q(str_list, substr, only_one=False):
    """
    Return True if `substr` (str) is a substring of any one of the items 
    in `str_list` list, or False otherwise.
    """
    n = n_substr_in_list(str_list, substr)
    if only_one & n > 1:
        raise Exception('Found more than one item in list with "' + substr + '" substring.')
    if n >= 1:
        return True
    else:
        return False

    
def load_tse_dir(data_dir, conv=None, verbose=False):
    """
    Given a directory `data_dir` with data from TSE and a file 
    called 'header.txt' containing column names if the TSE files 
    do not contain headers, load all files i the folder to a single
    Pandas DataFrame by concatenating them.
    """
    
    # Get file list in folder:
    file_list   = glob(data_dir + '*.txt') + glob(data_dir + '*.csv')
    txt_A = substr_in_list_Q(file_list, '.txt')
    csv_A = substr_in_list_Q(file_list, '.csv')
    # Sanity checks and messages:
    if csv_A and txt_A:
        raise Exception("Found both TXT and CSV files, don't know what to do.")
    if verbose:
        if txt_A:
            print('Found TXT files. Will require header file.')
        if csv_A:
            print('Found CSV files.')
            
    # Get list of data files:
    data_list = substr_in_list(file_list, 'header', not_in=True)

    # Filter data files for BRASIL one, if existent:
    brasil_list = substr_in_list(file_list, 'BRASIL')
    if len(brasil_list) > 1:
        raise Exception('Found more than one BRASIL data file.')
    if len(brasil_list) == 1:
        data_list = brasil_list
        if verbose:
            print('Found a full BRASIL file, will use it.')
    
    # Get header, if existent:
    header_A = substr_in_list_Q(file_list, 'header', only_one=True)
    if header_A:
        if verbose:
            print('Found header, will load it.')
        header = pd.read_csv(data_dir + 'header.txt', sep=';').columns
    else:
        header = None

    # Load data:
    if verbose:
        print('Loading data...')
    df_list = []
    for data_file in data_list:
        df = load_tse_file(data_file, header, conv)
        df_list.append(df)

    # Concatenate data:
    if verbose:
        print('Concatenating datasets...')
    df = pd.concat(df_list, ignore_index=True)

    return df


def set_diff(A, B):
    """
    Return a list of elements in set `A` that are not in set `B`.
    """
    return sorted(list(set(A) - set(B)))


def build_col_trafo(template_header, df_header, apply_replace=True):
    # Prefixes and terms from old files:
    hist_col_prefix     = ['DES_SITUACAO_CANDIDATURA', 'COD_SITUACAO_CANDIDATURA',
                           'CODIGO_', 'COD_', 'DESCRICAO_', 'DESC_', 'DES_', 'NOME_', 'NUMERO_', 'NUM_', 
                           'SIGLA_', '_SEXO', 'CPF_CANDIDATO', 'DESPESA_MAX_CAMPANHA', 'SEQUENCIAL_CANDIDATO', 
                           'COMPOSICAO_LEGENDA', 'CD_LEGENDA', 'NM_LEGENDA', 'DS_UE', 'HORA_', 
                           'DATA_GERACAO', 'DATA_NASCIMENTO', 'NR_NR_', 'VR_VR_', 'DATA_ULTIMA_ATUALIZACAO',
                           'DETALHE_BEM', 'VALOR_BEM', 'DS_SIT_CANDIDATO', 
                           'CD_SIT_CANDIDATO', '_SIT_CAND_SUPERIOR', '_SIT_CAND_TOT', 'SEQUENCIAL_LEGENDA',
                           'TRANSITO', 'TOTAL_VOTOS', 'NR_CAND', 'NR_CANDIDATOIDATO',
                           'ST_VOTO_EM_ST_VOTO_EM_TRANSITO']
    
    # Respective prefixes and terms for template:
    cand2020_col_prefix = ['DS_DETALHE_SITUACAO_CAND', 'CD_DETALHE_SITUACAO_CAND',
                           'CD_', 'CD_', 'DS_', 'DS_', 'DS_', 'NM_', 'NR_', 'NR_', 
                           'SG_', '_GENERO', 'NR_CPF_CANDIDATO', 'VR_DESPESA_MAX_CAMPANHA', 'SQ_CANDIDATO', 
                           'DS_COMPOSICAO_COLIGACAO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 'NM_UE', 'HH_', 
                           'DT_GERACAO', 'DT_NASCIMENTO', 'NR_', 'VR_', 'DT_ULTIMA_ATUALIZACAO',
                           'DS_BEM_CANDIDATO', 'VR_BEM_CANDIDATO', 'DS_DETALHE_SITUACAO_CAND',
                           'CD_DETALHE_SITUACAO_CAND', '_SITUACAO_CANDIDATURA', '_SIT_TOT_TURNO', 'SQ_COLIGACAO',
                           'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_NOMINAIS', 'NR_CANDIDATO', 'NR_CANDIDATO',
                           'ST_VOTO_EM_TRANSITO']
    
    if apply_replace:
        # Replace all olf prefixes and terms for new ones:
        new_hist_cols   = xu.mass_replace(df_header, hist_col_prefix, cand2020_col_prefix)
    else:
        new_hist_cols   = df_header
        
    # Get columns lost in template:
    extra_cols = set_diff(new_hist_cols, template_header)

    return new_hist_cols, extra_cols


def standardize_df_columns(df, template_header, old_cols):
    """
    Transform the `df` (DataFrame) header so it matches the `template_header`
    (list of str) appended with extra columns in `old_cols` (list of str). 
    """
    # Get df header:
    df_header = df.columns
    
    # Get new names for df columns and columns not present in template header:
    new_df_cols, extra_cols = build_col_trafo(template_header, df_header)
    # Build a dict to translate df column names to template style:
    col_mapper = dict(zip(df_header, new_df_cols))
    
    if len(set(extra_cols) - set(old_cols)) > 0:
        raise Exception('There are columns in DataFrame that are not accounted for.')
    
    # Create standard table header:
    template_frame = pd.DataFrame(data=[], columns=list(template_header) + old_cols)

    # Translate columns in df to template style:
    df = df.rename(col_mapper, axis=1)
    # Put df under standard header:
    df = pd.concat([template_frame, df])
    
    return df


def format_date(date):
    """
    Standardize format for (birth) dates that are in weird or inconsistent formats,
    specially in old databases.
    """
    # Para mudar formatação de data fora do padrão:
    MonthDic = {'JAN':'01', 'FEB':'02', 'MAR':'03', 'APR':'04', 'MAY':'05', 'JUN':'06', 
                'JUL':'07', 'AUG':'08', 'SEP':'09', 'OCT':'10', 'NOV':'11', 'DEC':'12'}
    MonthMax = {'01':31,'02':29,'03':31,'04':30,'05':31,'06':30,
                '07':31,'08':31,'09':30,'10':31,'11':30,'12':31}

    try:
        # Caso campo em branco:
        if type(date)==float and np.isnan(date):
            return ''
        # Caso campo contendo espaços:
        if date.find(' ')!=-1:
            if len(date)==7:
                date = date.replace(' ', '')
                togo = ['0'+date[0:1],'0'+date[1:2],date[2:]]
            else:
                togo = [str(int(date[:2])).zfill(2),str(int(date[2:4])).zfill(2),date[4:]]
        # Caso campo apenas números:
        elif date.isdigit()==True:
            if len(date)==6:
                if date[2:4]=='19':
                    togo = ['0'+date[0:1],'0'+date[1:2],date[2:]]
                else:
                    togo = [date[:2],date[2:4],'19'+date[4:]]
            elif len(date)==7:
                if date[3:5]=='19':
                    if int(date[1:3])>12:
                        togo = [date[:2],'0'+date[2:3],date[3:]]
                    else:
                        togo = ['0'+date[:1],date[1:3],date[3:]]
                elif date[4:6]=='19':
                    togo = [date[:2],date[2:4],date[4:]+'0']
                else:
                    print('Data ininteligível 1:', date)
                    return date
            else:
                togo = [date[:2],date[2:4],date[4:]]
            if int(togo[0])>MonthMax[togo[1]]:
                togo[0]=str(int(togo[0])-1)
            if togo[0]=='29' and togo[1]=='02' and (int(togo[2])-1900)%4!=0:
                togo[0]='28'
        # Caso separação por traços:
        elif date.find('-')!=-1:
            togo = date.split('-')
            if togo[1].isdigit()==False:
                togo[1] = MonthDic[togo[1]]
                if len(togo[2])==2:
                    togo[2] = '19'+togo[2]
                else:
                    print('Data ininteligível 2:', date)
            else:
                print('Data ininteligível 3:', date)
        # Caso separação por barra:
        else:
            togo = date.split('/')
        # Casos de ano apenas com dois dígitos:
        if int(togo[2])<1800 or int(togo[2])>2100:
            return togo[0]+'/'+togo[1]+'/19'+togo[2][2:]
        # Caso ano com 4 dígitos:
        else:
            return '/'.join(togo)
    
    except:
        return None

    
def clean_tit_eleitor(tit_eleitor_str):
    """
    Strip whitespaces from Título de eleitor `tit_eleitor_str` (str)
    and pad with zeros (to get 12 digits).
    """
    tit_eleitor_str = ''.join(tit_eleitor_str.split())
    tit_eleitor_str = tit_eleitor_str.rjust(12,'0')
    return tit_eleitor_str

def clean_cpf(cpf_series):
    """
    Apply hard-coded translation of CPFs to a Pandas Series `cpf_series`.
    The hard-coded dict was created based on the search for other entries 
    (in other years) with same name and Tit. eleitor.
    """
    cpf_fixer = {'48 110000':'53909542549', '60 30434549':'60630434549', '58 753000':'68411537404'}
    cpf_series = cpf_series.map(xu.translate_dict(cpf_fixer))
    return cpf_series


def clean_votos_df(df):
    """
    Clean and fix "votacao_candidato_munzona" dataframes.
    """
    
    # SG_UE for 2006 is wrong, fix it:
    df.loc[df['ANO_ELEICAO'] == 2006, 'SG_UE'] = df.loc[df['ANO_ELEICAO'] == 2006, 'SG_UF']
    
    # Standardize nome do cargo:
    df['DS_CARGO'] = df['DS_CARGO'].str.upper()
    return df


def clean_cand_df(df):
    """
    Clean and fix "consulta_cand" DataFrame.
    """
    # Fix birth dates manually:
    bad_date = ['3010196', '3010179', '2801 966', '29021954', '29021947']
    fix_date = ['30101960', '30101979', '28011966', '28021954', '28021947']
    for bad, fix in zip(bad_date, fix_date):
        df.loc[df['DT_NASCIMENTO'] == bad, 'DT_NASCIMENTO'] = fix
    # Fix birth dates with formula:
    df['DT_NASCIMENTO'] = df['DT_NASCIMENTO'].apply(format_date)

    # Fix level of education:
    df['DS_GRAU_INSTRUCAO'] = df['DS_GRAU_INSTRUCAO'].str.replace('^FUNDAMENTAL', 'ENSINO FUNDAMENTAL', regex=True)
    df['DS_GRAU_INSTRUCAO'] = df['DS_GRAU_INSTRUCAO'].str.replace('^MÉDIO', 'ENSINO MÉDIO', regex=True)

    # Clean documents (CPF and Tit Eleitor):
    df['NR_TITULO_ELEITORAL_CANDIDATO'] = df['NR_TITULO_ELEITORAL_CANDIDATO'].apply(clean_tit_eleitor)
    df['NR_CPF_CANDIDATO'] = clean_cpf(df['NR_CPF_CANDIDATO'])
    
    # Standardize descrição do cargo:
    df['DS_CARGO'] = df['DS_CARGO'].str.replace('º SUPLENTE SENADOR', 'º SUPLENTE')
    
    
    return df


def clean_bem_df(df):
    """
    Clean and fix "bem_candidato" DataFrame.
    """
    df['VR_BEM_CANDIDATO'] = df['VR_BEM_CANDIDATO'].str.replace(',', '.')
    df['VR_BEM_CANDIDATO'] = df['VR_BEM_CANDIDATO'].astype(float)
    # Some values are negative, but by the looks of it, this is a mistake:
    df['VR_BEM_CANDIDATO'] = df['VR_BEM_CANDIDATO'].abs()
    return df


def load_header_from_file(filename):
    """
    Load the header associated with a single TSE data file `filename` (str).
    """
    return load_tse_file(filename).columns


def load_header_from_dir(folder):
    """
    Load the header associated with a folder of TSE data files `folder` (str).
    """
    return load_tse_dir(folder).columns


def clear_all_null_tokens(df):
    """
    Replace all known TSE tokens that represent missing values with None.
    """
    for col in df.columns:
        df.loc[df[col].isin([-1, -3, '-1', '-3', '#NULO#', '#NE#', '#NI#', '00000000#NI#', '0000000000-1']), col] = None
    return df


def etl_cand(folder, template_header, destination):
    """
    Extract, Transform and Load TSE data about the candidates (consulta_cand).
    
    Input
    -----
    
    folder : str
        Path to the folder containing the data. For data files that do not contain headers
        (.txt files), the folder should also contain a 'header.txt' file with the header 
        in the same format as the data files.
        
    template_header : list of str
        The column names that come from a template. Those are most of the columns that 
        will appear in the output file (some others are hard-coded).
    
    destination : str
        Path to a single file that will receive all the data in `folder`.
    """
    
    # Hard-coded stuff:
    # Colunas extras (para além do template): 
    consulta_cand_old_cols = ['IDADE_DATA_ELEICAO', 'SG_LEGENDA']
    # How to read in certain columns:
    cand_types = {'DATA_NASCIMENTO':str, 'CPF_CANDIDATO':str, 'NUM_TITULO_ELEITORAL_CANDIDATO':str,
                  'NR_TITULO_ELEITORAL_CANDIDATO': str, 'NR_CPF_CANDIDATO': str}    
    
    # Load data from folder:
    df = load_tse_dir(folder, cand_types, verbose=False)
    df_header = df.columns
    
    # Standardize CONSULTA CAND:
    trans = standardize_df_columns(df, template_header, consulta_cand_old_cols)
    trans = clean_cand_df(trans)
    trans = clear_all_null_tokens(trans)
    
    # Save file:
    xu.make_necessary_dirs(destination)
    trans.to_csv(destination, index=False, quoting=csv.QUOTE_ALL)
    

def etl_bem(folder, template_header, destination):
    """
    Extract, Transform and Load TSE data about the candidates' wealth (bem_candidato).
    
    Input
    -----
    
    folder : str
        Path to the folder containing the data. For data files that do not contain headers
        (.txt files), the folder should also contain a 'header.txt' file with the header 
        in the same format as the data files.
        
    template_header : list of str
        The column names that come from a template. Those are most of the columns that 
        will appear in the output file (some others are hard-coded).
    
    destination : str
        Path to a single file that will receive all the data in `folder`.
    """
    
    # Hard-coded stuff:
    # Colunas extras (para além do template): 
    bem_candidato_old_cols = []
    # How to read in certain columns:
    bem_types  = {'VALOR_BEM':str, 'VR_BEM_CANDIDATO':str}
    
    # Load data from folder:
    print('Load data...')
    df = load_tse_dir(folder, bem_types, verbose=False)
    df_header = df.columns
    
    # Standardize CONSULTA CAND:
    print('Standardize...')
    trans = standardize_df_columns(df, template_header, bem_candidato_old_cols)
    print('Clean...')
    trans = clean_bem_df(trans)
    print('Clear nulls...')
    trans = clear_all_null_tokens(trans)
    
    # Save file:
    print('Save file...')
    xu.make_necessary_dirs(destination)
    trans.to_csv(destination, index=False, quoting=csv.QUOTE_ALL)


def etl_votos(infile, template_header, destination):
    """
    Extract, Transform and Load TSE data about the candidate's votes (votacao_candidato_munzona).
    
    Input
    -----
    
    infile : str
        Path to the data file. For data files that do not contain headers
        (.txt files), the folder containing the file should also contain a 'header.txt' 
        file with the header in the same format as the data files.
        
    template_header : list of str
        The column names that come from a template. Those are most of the columns that 
        will appear in the output file (some others are hard-coded).
    
    destination : str
        Path to a file that will receive all the  cleaned data from `infile`.
    """
    
    # Hard-coded stuff:
    # Colunas extras (para além do template): 
    votacao_old_cols = []
    
    # Load data from folder:
    df = load_tse_file(infile, verbose=False)
    df_header = df.columns
    
    # Standardize CONSULTA CAND:
    trans = standardize_df_columns(df, template_header, votacao_old_cols)
    trans = clean_votos_df(trans)
    trans = clear_all_null_tokens(trans)
    
    # Save file:
    xu.make_necessary_dirs(destination)
    trans.to_csv(destination, index=False, quoting=csv.QUOTE_ALL)

## Limpa dados do TSE

### Candidatos

In [3]:
cand_header = load_header_from_dir('/home/skems/ceweb/dados/brutos/tse/consulta_cand_2022/')
etl_cand('/home/skems/ceweb/dados/brutos/tse/consulta_cand_2022/', cand_header, '/home/skems/ceweb/dados/limpos/tse/consulta_cand/consulta_cand_2022_BRASIL.csv')

### Bens

In [4]:
bem_header = load_header_from_dir('/home/skems/ceweb/dados/brutos/tse/bem_candidato_2022/')
etl_bem('/home/skems/ceweb/dados/brutos/tse/bem_candidato_2022/', bem_header, '/home/skems/ceweb/dados/limpos/tse/bens_candidatos/bem_candidato_2022_BRASIL.csv')

Load data...
Standardize...
Clean...
Clear nulls...
Save file...


#### Teste dos dados

In [5]:
import xavy.explore as xe

In [6]:
bens_df = pd.read_csv('/home/skems/ceweb/dados/limpos/tse/bens_candidatos/bem_candidato_2022_BRASIL.csv')

In [7]:
xe.checkMissing(bens_df)

Colunas com valores faltantes:
Empty DataFrame
Columns: [coluna, N, %]
Index: []


In [8]:
xe.mapUnique(bens_df)


DT_GERACAO: 1 unique values.
16/08/2022

HH_GERACAO: 1 unique values.
06:07:10

ANO_ELEICAO: 1 unique values.
2022

CD_TIPO_ELEICAO: 1 unique values.
2

NM_TIPO_ELEICAO: 1 unique values.
Eleição Ordinária

CD_ELEICAO: 2 unique values.
544,  546

DS_ELEICAO: 2 unique values.
Eleição Geral Federal 2022,  Eleições Gerais Estaduais 2022

DT_ELEICAO: 1 unique values.
02/10/2022

SG_UF: 28 unique values.
(sample) AL,  BA,  BR,  CE,  DF,  ES,  GO,  MA,  MG,  MS,  MT,  PA,  PB,  PE,  PR,  RR,  RS,  SE,  SP,  TO

SG_UE: 28 unique values.
(sample) AL,  AM,  BA,  BR,  CE,  ES,  GO,  MA,  MS,  MT,  PA,  PB,  PE,  PI,  RJ,  RS,  SC,  SE,  SP,  TO

NM_UE: 28 unique values.
(sample) ACRE,  ALAGOAS,  AMAPÁ,  AMAZONAS,  BRASIL,  DISTRITO FEDERAL,  ESPÍRITO SANTO,  GOIÁS,  MINAS GERAIS,  PARANÁ,  PARAÍBA,  PERNAMBUCO,  PIAUÍ,  RIO DE JANEIRO,  RIO GRANDE DO NORTE,  RIO GRANDE DO SUL,  RORAIMA,  SANTA CATARINA,  SÃO PAULO,  TOCANTINS

SQ_CANDIDATO: 17657 unique values.
(sample) 50001604575,  50001605371

## Outros dados (código antigo)

In [ ]:
# ETL votos:

header_file  = '../dados/brutos/votacao_candidato_munzona_2018/votacao_candidato_munzona_2018_AC.csv'
votos_header = load_header_from_file(header_file)

for year in range(2004, 2022, 2):
    print(year)
    file_list = sorted(glob('../dados/brutos/votacao_candidato_munzona_' + str(year) + '/votacao_candidato_munzona_' + str(year) + '_??.*'))
    for f in file_list:
        uf = f[-6:-4]
        print(uf, end=' ')
        
        outfile = '../dados/limpos/votacao_candidato_munzona/' + f.split('/')[-1].replace('.txt', '.csv')
        etl_votos(f, votos_header, outfile)
    
    print('')

2020
AC AL AM AP BA CE ES GO MA MG MS MT PA PB PE PI PR RJ RN RO RR RS SC SE SP TO 


In [6]:
# ETL bens dos candidatos:

bem_header = load_header_from_dir('../dados/brutos/bem_candidato_2020/')

for year in range(2006, 2022, 4):
    print(year)
    in_folder = '../dados/brutos/bem_candidato_' + str(year) + '/'
    out_file  = '../dados/limpos/bem_candidato/bem_candidato_' + str(year) + '_BRASIL.csv'
    etl_bem(in_folder, bem_header, out_file)

2020
Load data...
Standatdize...
Clean...
Clear nulls...
Save file...
